# Lesson 3. Land Cover Classification
# Land Cover Classification using Lucas 2018 and GeoCubes with Scicit-Learn and Dask.


## Goal 

 

In this exercise, you will perform a land cover classification for some Finnish region using [Lucas 2018](https://ckan.ymparisto.fi/en/dataset/lucas2018) inventory dataset and geospatial data from the [GeoCubes Finland](https://vm0160.kaj.pouta.csc.fi/geocubes/) service. After the classification, you will analyse the accuracy of the result and the relevance of the data you selected to use in this task. 

You will learn:
 * How to conduct a generalized land cover classification using the scikit-learn Python library
 * How to use Dask framework in CSC's Puhti supercomputer
 * How to interpret the classification results and improve them

## Background

This type of land cover interpretation provides the background for various environmental monitoring applications, such as the calculation of environmental indicators (e.g. "land degradation"), biotope inventories, nutrient loss modelling, assessment of the consequences of legislation or land use planning.  

The Finnish Environment Institute has long since the 1990s recognised the importance of such up-to-date spatial data, without forgetting the changes. SYKE has participated in the [European land cover monitoring](https://land.copernicus.eu/) since the early 2000s within the Corine programme, which has produced Corine land cover interpretations and modifications for the years 2000, 2006, 2012 and 2018. In addition, more local interpretations have been made with various partners, such as the remote sensing of Northern Lapland carried out for Metsähallitus in recent years. There are also various projects, such as FEO and Geoportti, which aim to improve the availability of environmental spatial information. 

More infromation: 

https://www.syke.fi/fi-FI/Avoin_tieto/Seurantatiedot/Maanpeitteen_seuranta 

 
## Input-data: 

1. **In-situ** 

    In this exercise the In-situ dataset is the Lucas 2018 land inventory dataset from Finland. LUCAS (Land USe and Coverage Area frame Survey) is a land inventory organised by EUROSTAT, the statistical office of the European Union, which collects data on land cover and land use from the survey plots. There are about 330,000 plots throughout the EU, of which about 17,000 are in Finland. Part of the data collection is done by field visits and part by interpreting aerial photographs.  



    The link https://a3s.fi/geoportti_training/lucas2018.tar provides Lucas 2018 scores for the whole country (lucas_2018.shp) and by province, with the main land cover group found in the column "lc_class": 

    1 = Build up areas 

    2 = Agriculture 

    3 = Forests

    4 = Sparsely vegetated forests |

    5 = Grasslands 

    6 = Unvegetated soil 

    7 = Water 

    8 = Wetlands


    A more detailed land cover class can be found in column LC1 (for more detailed information on land cover and use classes see https://ec.europa.eu/eurostat/documents/205002/8072634/LUCAS2018-C3-Classification.pdf if needed. 



 2. **Features**

 
    In this exercise, you will use raster datasets found in the [GeoCubes Finland](https://vm0160.kaj.pouta.csc.fi/geocubes/) -service as classification features.

    So, select a province and download the data of your choice, delimited by that province, using a pixel size of 10 m


 



 

 

### Import neccesary libraries

In [3]:
import rasterio
import rioxarray as rxr
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os
import urllib
import multiprocessing
from dask import delayed
from dask import compute
from dask.diagnostics import ProgressBar
from dask.distributed import Client
import dask
import dask.array as da
from sklearn.preprocessing import LabelEncoder
from pprint import pprint
from tqdm.notebook import tqdm
import itertools

## Set variables for dask

In [4]:
# Dask related environment variables
%env MALLOC_TRIM_THRESHOLD_=4
%env OMP_NUM_THREADS=4
%env MKL_NUM_THREADS=4
%env OPENBLAS_NUM_THREADS=4

env: MALLOC_TRIM_THRESHOLD_=4
env: OMP_NUM_THREADS=4
env: MKL_NUM_THREADS=4
env: OPENBLAS_NUM_THREADS=4


In [3]:
processes = False
threads_per_worker = None

## Create a Dask cluster and client

In [5]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:43817")
client

<Client: 'tcp://127.0.0.1:43817' processes=4 threads=4, memory=75.00 GiB>

## Load the reference data from Allas

In [7]:
!mkdir /users/jutilaee/land_cover_classification
!wget -P /users/jutilaee/land_cover_classification https://a3s.fi/geoportti_training/lucas2018.tar

mkdir: cannot create directory ‘/users/jutilaee/land_cover_classification’: File exists
--2023-09-04 12:56:12--  https://a3s.fi/geoportti_training/lucas2018.tar
Resolving a3s.fi (a3s.fi)... 86.50.254.19, 86.50.254.18
Connecting to a3s.fi (a3s.fi)|86.50.254.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65228800 (62M) [application/x-tar]
Saving to: ‘/users/jutilaee/land_cover_classification/lucas2018.tar.2’

lucas2018.tar.2     100%[===================>]  62.21M   288MB/s    in 0.2s    

2023-09-04 12:56:12 (288 MB/s) - ‘/users/jutilaee/land_cover_classification/lucas2018.tar.2’ saved [65228800/65228800]



In [8]:
!cd /users/jutilaee/land_cover_classification
!tar -xf /users/jutilaee/land_cover_classification/lucas2018.tar -C /users/jutilaee/land_cover_classification

In [12]:
lucas2018_lapland = gpd.read_file('/users/jutilaee/land_cover_classification/training_data/lucas2018_19_lappi.shp')
lucas2018_lapland.columns

Index(['OBJECTID', 'POINT_ID', 'OFFICE_PI', 'SURVEYDATE', 'OBS_DIST',
       'OBS_DIRECT', 'OBS_TYPE', 'LC1', 'LC2', 'LU1', 'LU2', 'LC1_name',
       'LU1_name', 'photolinkN', 'photolinkE', 'photolinkS', 'photolinkW',
       'photolinkP', 'LCluokka', 'maakunta_1', 'geometry'],
      dtype='object')

## Load input rasters from GeoCubes

In [10]:
def download_data(params):
    download_url = params[0]
    out_fn = params[1]
    print(f"Starting to download {download_url} \n")
    r = urllib.request.urlretrieve(download_url, out_fn)
    print(f"Download completed. Results saved to {out_fn}")

In [11]:
raster_list = [
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/ndvimax/maakuntajako:19/2021",
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/km2/maakuntajako:19/2022",
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/puustoisuusluokat/maakuntajako:19/2022",
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/kasvillisuudenkorkeus/maakuntajako:19/2022",
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/latvuspeitto/maakuntajako:19/2022",
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/sentinel2-rgb/maakuntajako:19/2021",
    "https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/sentinel2-nir/maakuntajako:19/2021"]
    
outfp = ["/scratch/project_2000630/jutilaee/geoportti/input_data/ndvi_max_2021_lapland.tif",
         "/scratch/project_2000630/jutilaee/geoportti/input_data/km2_lapland.tif",
         "/scratch/project_2000630/jutilaee/geoportti/input_data/tree_classes_lapland.tif",
         "/scratch/project_2000630/jutilaee/geoportti/input_data/vegetation_height_lapland.tif",
         "/scratch/project_2000630/jutilaee/geoportti/input_data/canopy_cover_lapland.tif",
         "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland.tif",
         "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_nir_lapland.tif"]

In [12]:
param_list = []

for i in range(len(raster_list)):
	param_list.append([raster_list[i], outfp[i]])

In [13]:
#param_list

In [14]:
list_of_delayed_functions = []

for parameters in param_list:
    list_of_delayed_functions.append(delayed(download_data)(parameters))


In [16]:
#download_data(["https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/sentinel2-rgb/maakuntajako:19/2021", "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland.tif"])


In [17]:
%%time
with ProgressBar():
        compute(list_of_delayed_functions)

Starting to download https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/latvuspeitto/maakuntajako:19/2022 

Starting to download https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/sentinel2-nir/maakuntajako:19/2021 

Starting to download https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/puustoisuusluokat/maakuntajako:19/2022 

Starting to download https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/sentinel2-rgb/maakuntajako:19/2021 

Download completed. Results saved to /scratch/project_2000630/jutilaee/geoportti/input_data/tree_classes_lapland.tif
Starting to download https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/km2/maakuntajako:19/2022 

Download completed. Results saved to /scratch/project_2000630/jutilaee/geoportti/input_data/canopy_cover_lapland.tif
Starting to download https://vm0160.kaj.pouta.csc.fi/geocubes/clip/10/ndvimax/maakuntajako:19/2021 

Download completed. Results saved to /scratch/project_2000630/jutilaee/geoportti/input_data/ndvi_max_2021_lapland.tif
Starting to downloa

## Rasterize training data


In [7]:
%%time
! gdal_rasterize /users/jutilaee/land_cover_classification/training_data/lucas2018_19_lappi.shp /scratch/project_2000630/jutilaee/geoportti/lucas2018_lapland.tif -ot "UInt16" \
    -a LCluokka \
    -tr 10 10 \
    -te 243095.91 7244045.217 627855.91 7776445.217 -co "COMPRESS=LZW"

0...10...20...30...40...50...60...70...80...90...100 - done.
CPU times: user 125 ms, sys: 30.3 ms, total: 156 ms
Wall time: 16 s


## Split multi-band rasters to single-band rasters

In [21]:
%%time
def split_multiband_raster(input_path, output_folder):
    # Open the multiband raster
    ds = rxr.open_rasterio(input_path)

    # Get the number of bands
    num_bands = ds.shape[0]

    # Loop through each band
    for band_index in range(num_bands):
        # Get the band data
        band = ds[band_index]

        # Create the output file name
        output_name = f"s2_rgb_lapland_band{band_index + 1}.tif"
        output_path = os.path.join(output_folder, output_name)

        # Save the band as a single-band raster
        band.rio.to_raster(output_path)

        print(f"Band {band_index + 1} saved as {output_path}")

# Example usage
input_raster = "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland.tif"
output_folder = "/scratch/project_2000630/jutilaee/geoportti/input_data"
split_multiband_raster(input_raster, output_folder)

Band 1 saved as /scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland_band1.tif
Band 2 saved as /scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland_band2.tif
Band 3 saved as /scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland_band3.tif


## Create a datacube

In [11]:
%%time
rasters = [
    "/scratch/project_2000630/jutilaee/geoportti/input_data/ndvi_max_2021_lapland.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/km2_lapland.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/tree_classes_lapland.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/vegetation_height_lapland.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/canopy_cover_lapland.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland_band1.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland_band2.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_rgb_lapland_band3.tif",
    "/scratch/project_2000630/jutilaee/geoportti/input_data/s2_nir_lapland.tif"
]


band_names = [
    'ndvi_max', 'dem', 'tree_classes', 'veg_height', 'canopy_cover',
    's2_b4', 's2_b3', 's2_b2', 's2_nir'
]

def load_raster(raster, band_name):
    xds = rxr.open_rasterio(raster)
    return xds.rename(band_name)

delayed_rasters = [
    dask.delayed(load_raster)(raster, band_names[i]) for i, raster in enumerate(rasters)
]

rasters_to_merge = dask.compute(*delayed_rasters)

cube = xr.merge(rasters_to_merge)
cube = cube.transpose('band', 'y', 'x')
cube


CPU times: user 30.3 ms, sys: 3.31 ms, total: 33.6 ms
Wall time: 134 ms


<xarray.Dataset>
Dimensions:       (band: 1, x: 38477, y: 53241)
Coordinates:
  * band          (band) int64 1
  * x             (x) float64 2.431e+05 2.431e+05 ... 6.278e+05 6.279e+05
  * y             (y) float64 7.776e+06 7.776e+06 ... 7.244e+06 7.244e+06
    spatial_ref   int64 0
Data variables:
    ndvi_max      (band, y, x) uint8 ...
    dem           (band, y, x) float32 ...
    tree_classes  (band, y, x) uint8 ...
    veg_height    (band, y, x) uint8 ...
    canopy_cover  (band, y, x) uint8 ...
    s2_b4         (band, y, x) uint16 ...
    s2_b3         (band, y, x) uint16 ...
    s2_b2         (band, y, x) uint16 ...
    s2_nir        (band, y, x) uint16 ...
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

## Export the datacube to a GeoTIFF
#### Needs a lot of memory, not sure how much! Could this one be skipped?

In [ ]:
%%time


if not os.path.exists("/scratch/project_2000630/jutilaee/geoportti/datacubes"):
    os.mkdir("/scratch/project_2000630/jutilaee/geoportti/datacubes")
    
# Save the datacube to GeoTIFF
#with dask.config.set(scheduler='single-threaded'):
cube.rio.to_raster("/scratch/project_2000630/jutilaee/geoportti/datacubes/lappi.tiff")

## Sample raster

In [18]:
gdf = lucas2018_lapland

In [19]:
%%time
import pandas as pd
import geopandas as gpd
import xarray as xr

# Assuming 'cube' is your xarray dataset containing multiple bands

# Define x and y coordinates
da_x = xr.DataArray(gdf.geometry.x.values, dims=['z'])
da_y = xr.DataArray(gdf.geometry.y.values, dims=['z'])
lc_class = xr.DataArray(gdf.LCluokka.values, dims=['z'])


# Define band names
band_names = [
    'ndvi_max', 'dem', 'tree_classes', 'veg_height', 'canopy_cover',
    's2_b4', 's2_b3', 's2_b2', 's2_nir'
]

# Select values from different bands at the nearest location
band_values = cube.sel(x=da_x, y=da_y, method='nearest')

# Create a dataframe with values from different bands and corresponding coordinates
results = pd.DataFrame({
    band_name: band_values[band_name].values.flatten()
    for band_name in band_names
})
results['x'] = da_x.values.flatten()
results['y'] = da_y.values.flatten()

# Create a GeoDataFrame with the dataframe and geometry
results['geometry'] = gpd.points_from_xy(results.x, results.y)
results = gpd.GeoDataFrame(results)
results['lc_class'] = lc_class

# Print the results
results



CPU times: user 34.8 s, sys: 16.2 s, total: 51 s
Wall time: 2min 33s


,ndvi_max,dem,tree_classes,veg_height,canopy_cover,s2_b4,s2_b3,s2_b2,s2_nir,x,y,geometry,lc_class
0,112,859.723877,30,255,127,1047,977,841,1500,283350.7473,7.644968e+06,POINT (283350.747 7644968.254),6
1,113,778.580078,30,255,127,1135,1103,995,1510,283893.0358,7.646917e+06,POINT (283893.036 7646916.661),6
2,142,699.859863,30,255,127,743,730,600,2150,287148.1397,7.658608e+06,POINT (287148.140 7658607.672),5
3,178,197.963943,11,4,15,224,423,271,3281,357658.6338,7.362535e+06,POINT (357658.634 7362534.738),3
4,167,151.150208,11,3,2,267,429,311,2159,363490.3585,7.383922e+06,POINT (363490.359 7383921.752),3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593,143,190.248138,11,0,0,439,428,343,1717,519711.4625,7.683349e+06,POINT (519711.463 7683348.704),8
2594,160,237.982376,30,255,127,491,561,346,2536,538993.3701,7.715970e+06,POINT (538993.370 7715969.992),5
2595,155,173.440186,11,47,20,383,468,328,1888,525993.0993,7.683817e+06,POINT (525993.099 7683817.091),3
2596,156,226.751831,30,0,0,625,613,399,2489,551216.9449,7.737891e+06,POINT (551216.945 7737890.763),8


In [20]:
df= results.drop(['x','y','geometry'], axis=1)
df

,ndvi_max,dem,tree_classes,veg_height,canopy_cover,s2_b4,s2_b3,s2_b2,s2_nir,lc_class
0,112,859.723877,30,255,127,1047,977,841,1500,6
1,113,778.580078,30,255,127,1135,1103,995,1510,6
2,142,699.859863,30,255,127,743,730,600,2150,5
3,178,197.963943,11,4,15,224,423,271,3281,3
4,167,151.150208,11,3,2,267,429,311,2159,3
...,...,...,...,...,...,...,...,...,...,...
2593,143,190.248138,11,0,0,439,428,343,1717,8
2594,160,237.982376,30,255,127,491,561,346,2536,5
2595,155,173.440186,11,47,20,383,468,328,1888,3
2596,156,226.751831,30,0,0,625,613,399,2489,8


In [21]:
dfY = df.iloc[:,-1]
dfX = df.iloc[:,0:-1]

In [22]:
xds = cube

# Print info
print("Columns. Last one is chosen as target")
print("Index\t\tColumn")
for i, col in enumerate(df.columns):
    print(f"{i}\t\t{col}")
print()

print("\nTarget class distribution")
print("label\tcount")
print(dfY.value_counts())
print()

# Classes smaller than 6 are removed
drop_classes = dfY.value_counts()[dfY.value_counts()<6].index.values
drop_series = ~dfY.isin(drop_classes)

dfY = dfY.loc[drop_series]
dfX = dfX.loc[drop_series,:]

print("Classes smaller than 6 are removed:")
print(drop_classes)
print()

# Final dataset
X = dfX.to_numpy()
y = dfY.to_numpy()

le = LabelEncoder()
y = le.fit_transform(y)

print(f"Shape of X: {X.shape}")
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print('label mapping:')
pprint(le_name_mapping)

Columns. Last one is chosen as target
Index		Column
0		ndvi_max
1		dem
2		tree_classes
3		veg_height
4		canopy_cover
5		s2_b4
6		s2_b3
7		s2_b2
8		s2_nir
9		lc_class


Target class distribution
label	count
3    1271
8     741
4     235
5     194
0      52
6      42
7      38
2      17
1       8
Name: lc_class, dtype: int64

Classes smaller than 6 are removed:
[]

Shape of X: (2598, 9)
label mapping:
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8}


## Classification

In [23]:
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                            cohen_kappa_score,
                            make_scorer)
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from pprint import pprint

from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import dask.array as da

In [24]:
# Ranfom forest validation
import sklearn.ensemble as ensemble
seed = 42

skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=seed)

scoring= {'accuracy': make_scorer(accuracy_score),
          'f1': make_scorer(f1_score, zero_division=0, average='weighted'),
          'precision': make_scorer(precision_score, zero_division=0, average='weighted'),
          'kappa': make_scorer(cohen_kappa_score)
         }

clf = RandomForestClassifier()
scores = cross_validate(clf, X, y, 
                     scoring=scoring,
                     cv=skf,
                     return_estimator=True,
                     verbose=2,
                     n_jobs=-1)
print(clf)
scores = pd.DataFrame(scores).drop(['fit_time', 'score_time', 'estimator'],axis=1)
print(scores.describe().loc[['mean'],:])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


RandomForestClassifier()
      test_accuracy   test_f1  test_precision  test_kappa
mean       0.716321  0.685824        0.672686    0.549844


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    2.3s finished


In [25]:
scores

,test_accuracy,test_f1,test_precision,test_kappa
0,0.700000,0.663940,0.644253,0.522734
1,0.721154,0.690110,0.676305,0.554374
2,0.721154,0.686028,0.676207,0.552944
3,0.726397,0.704036,0.692786,0.576095
4,0.722543,0.690606,0.680507,0.556155
5,0.721154,0.693050,0.676177,0.559049
6,0.700000,0.670682,0.658271,0.521294
7,0.730769,0.704874,0.694945,0.579485
8,0.710983,0.680055,0.664397,0.542199
9,0.709056,0.674861,0.663007,0.534117


array([[  13,    0,    0,   24,    0,    7,    1,    1,    6],
       [   2,    0,    0,    5,    0,    0,    0,    0,    1],
       [   0,    0,    0,    2,    0,   13,    0,    0,    2],
       [   4,    0,    0, 1143,   16,   11,    2,    2,   93],
       [   1,    0,    0,   79,   27,   22,    8,    0,   98],
       [   6,    0,    3,   53,   23,   36,    2,    1,   70],
       [   4,    0,    0,    7,    6,    2,   16,    1,    6],
       [   0,    0,    0,    3,    0,    1,    0,   31,    3],
       [   1,    0,    0,  107,   33,   16,    3,    0,  581]])

[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s


In [30]:
from dask_ml.wrappers import ParallelPostFit

# Choose the better model here by commenting the worse out
clf = ParallelPostFit(estimator=RandomForestClassifier(n_estimators=2000, random_state=0, n_jobs=-1))
# clf = ParallelPostFit(estimator=exported_pipeline)

clf.fit(X, y)

ParallelPostFit(estimator=RandomForestClassifier(n_estimators=2000, n_jobs=-1,
                                                 random_state=0))

In [31]:

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = cross_val_predict(clf, X, y, cv=5)

from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_pred)

array([[  14,    1,    0,   24,    0,    6,    1,    1,    5],
       [   2,    0,    0,    4,    0,    1,    0,    0,    1],
       [   0,    0,    0,    4,    0,   10,    0,    0,    3],
       [   6,    0,    0, 1147,   14,   12,    1,    0,   91],
       [   0,    0,    0,   82,   19,   23,   10,    0,  101],
       [   8,    0,    2,   48,   14,   39,    4,    1,   78],
       [   3,    0,    0,    4,    3,    4,   19,    1,    8],
       [   0,    0,    0,    3,    0,    1,    0,   31,    3],
       [   1,    0,    0,  106,   26,   25,    4,    0,  579]])

In [16]:
clf_base = clf.estimator
classes = le.inverse_transform(clf_base.classes_)
clf_base

RandomForestClassifier(n_estimators=2000, n_jobs=-1, random_state=0)

In [17]:
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)

In [18]:
clf2

ParallelPostFit(estimator=RandomForestClassifier(n_estimators=2000, n_jobs=-1,
                                                 random_state=0))

## Inference

These scripts by Mikko Impiö (SYKEcould be used https://github.com/sykefi/point-eo/blob/main/docs/demo_notebook.ipynb

Latest idea:
 1. Export the previously created model
 2. Save the datacube as a geotiff
 3. Use these POINT-EO scripts by by Mikko Impiö (SYKE) to get the classification map of the AOI: https://github.com/sykefi/point-eo/blob/main/docs/demo_notebook.ipynb
 
 ### Extra task for students (and us) to solve: How to feed the datacube directly to POINT-EO without exporting it to GeoTiff first?

In [ ]:
cube.data_vars.keys()

KeysView(Data variables:
    ndvi_max      (band, y, x) uint8 ...
    dem           (band, y, x) float32 ...
    tree_classes  (band, y, x) uint8 ...
    veg_height    (band, y, x) uint8 ...
    canopy_cover  (band, y, x) uint8 ...
    s2_b4         (band, y, x) uint16 ...
    s2_b3         (band, y, x) uint16 ...
    s2_b2         (band, y, x) uint16 ...
    s2_nir        (band, y, x) uint16 ...)

In [ ]:
numpy_arrays = np.array(numpy_arrays)

In [ ]:
numpy_arrays = numpy_arrays.squeeze(axis=1)


AxisError: axis 1 is out of bounds for array of dimension 1

In [53]:
numpy_arrays = numpy_arrays.reshape(-1, 9)
numpy_arrays

AttributeError: 'tuple' object has no attribute 'reshape'

In [ ]:
# Predict on whole image and save it as .tif file
# Credits: https://github.com/csc-training/geocomputing/blob/master/machineLearning/02_shallows/05_classification.ipynb
def predictImage(modelName, predictImage):
    #Set file paths for input and output files
    predictedClassesFile = outputImageBase + modelName + '.tif'
    predictedClassesPath = os.path.join(base_folder, predictedClassesFile)
    
    # Read the satellite image
    with rasterio.open(predictImage, 'r') as image_dataset:
        start_time = time.time()    
        
        #Reshape data to 1D as we did before model training
        #image_data = image_dataset.read()
        image_data = image_dataset.read()
        image_data2 = np.transpose(image_data, (1, 2, 0))
        #pixels = image_data2.reshape(-1, 22)
        pixels = image_data2.reshape(-1, 27)
        #Load the model from the saved file
        modelFilePath = os.path.join(base_folder, ('model_' + modelName + '.sav'))
        trained_model = load(modelFilePath)
        
        # predict the class for each pixel
        prediction = trained_model.predict(pixels)
        
        # Reshape back to 2D
        print('Prediction shape in 1D: ', prediction.shape)
        prediction2D = np.reshape(prediction, (image_dataset.meta['height'], image_dataset.meta['width']))
        #prediction2D = np.reshape(prediction, ( 705, 1213))
        print('Prediction shape in 2D: ', prediction2D.shape)
        
        # Save the results as .tif file.
        # Copy the coorindate system information, image size and other metadata from the satellite image 
        outputMeta = image_dataset.meta
        
        # Change the number of bands and data type.
        #outputMeta.update(count=1, dtype='uint8')
        outputMeta.update(count=1, dtype='uint8', nodata=255)
        # Writing the image on the disk
        with rasterio.open(predictedClassesPath, 'w', **outputMeta) as dst:
            dst.write(prediction2D, 1)
        plt.imshow(prediction2D)
        print('Predicting took: ', round((time.time() - start_time), 1), ' seconds')
        
        
        
        

In [17]:
import argparse
from pathlib import Path
import subprocess

import pickle
import numpy as np
import shapely
import rioxarray
import xarray as xr
import geopandas as gpd
from tqdm import tqdm
import dask.array as da
import dask
from dask.diagnostics import ProgressBar

from rioxarray.exceptions import NoDataInBounds

new_3d_xda = lambda c, d: xr.DataArray(
    c,
    name="classification",
    coords={"class": np.arange(c.shape[0]), "y": d.y, "x": d.x},
    dims=("class", "y", "x"),
)
new_2d_xda = lambda c, d: xr.DataArray(c, coords={"y": d.y, "x": d.x}, dims=("y", "x"))


def save_raster(x, name, crs):
    if not da.all(x == da.zeros_like(x)):
        x.rio.to_raster(name, compress="LZW", crs=crs, tiled=True, windowed=True)


# def process_xarray(Ax, tf):
#     T = torch.Tensor(da.asarray(Ax).compute())
#     T = tf(T)
#     return T

# def img2batch(sample, k):
#     pad = k//2
#     C,H,W = sample.shape

#     unfold = torch.nn.Unfold(kernel_size=(k,k), padding=pad)
#     UF = unfold(sample.unsqueeze(0))
#     B = torch.stack([UF[0,:,i].reshape((C,k,k)) for i in range(UF.shape[2])])

#     return B


def batch2img(sample, shape):
    H, W = shape
    return sample.reshape(H, W, sample.shape[1]).permute(2, 0, 1)


# def classify_block(T, model, k):
#     device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     C,H,W = T.shape
#     B = img2batch(T, k=k)
#     B = B.to(device)

#     with torch.no_grad():
#         out = model(B).softmax(1)
#     outimg = batch2img(out, (H,W))
#     return outimg


def full_inference_numpy(A, clf):
    # Reshaping
    A0 = np.moveaxis(A, 0, 2)
    ny, nx, chan = A0.shape
    a = A0.reshape(ny * nx, chan)

    # Classification
    c = clf.predict_proba(a)

    # Inverse reshaping
    C = c.reshape(ny, nx, -1)
    C = np.moveaxis(C, 2, 0)

    return C


def add_args(subparser):
    parser = subparser.add_parser("predict")
    parser.add_argument(
        "--model", type=str, required=True, help="Location of pickled model"
    )

    parser.add_argument("--input_raster", type=str, required=True)

    parser.add_argument("--cell_size", type=int)

    parser.add_argument("--block_buffer", type=int, help="block buffer in meters")

    parser.add_argument("--bit_depth", type=int, default=8)

    parser.add_argument("--extent", type=str, required=False)
    parser.add_argument("--out_folder", type=str, required=True)
    parser.add_argument(
        "--start_index", type=int, help="Starts processing from here in case of a crash"
    )
    parser.add_argument(
        "--crs", type=str, required=False, default="EPSG:3067", help="CRS for outputs"
    )
 

In [29]:
#input_file = Path(args.input_raster)
#model_file = Path(args.model)
out_folder = "/scratch/project_2000630/jutilaee/geoportti/"
#out_folder.mkdir(exist_ok=True, parents=True)
out_final = "/scratch/project_2000630/jutilaee/geoportti/"

# Model
#print(f"Using model {args.model}")
#with open(args.model, "rb") as f:
#    model = pickle.load(f)

model = clf2
print(model)

block_buffer = 5

# Raster
chunk_s = 2**10
"""
Fx = rioxarray.open_rasterio(
    args.input_raster,
    chunks={"band": -1, "x": chunk_s, "y": chunk_s},
    lock=False,
    parallel=True,
)
"""

Fx = cube

# Make grid
xmin = Fx.x.min()
ymin = Fx.y.min()
xmax = Fx.x.max()
ymax = Fx.y.max()

ParallelPostFit(estimator=RandomForestClassifier(n_estimators=2000, n_jobs=-1,
                                                 random_state=0))


In [30]:
cell_size = 1000
# projection of the grid
crs = "EPSG:3067"
# create the cells in a loop
grid_cells = []
for x0 in np.arange(xmin, xmax + cell_size, cell_size):
    for y0 in np.arange(ymin, ymax + cell_size, cell_size):
        # bounds
        x1 = x0 - cell_size
        y1 = y0 + cell_size
        grid_cells.append(shapely.geometry.box(x0, y0, x1, y1))
        
        

NameError: name 'shapely' is not defined

In [37]:
cell = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs=crs)
cell = cell.buffer(block_buffer, cap_style=3, join_style=2)

cell.to_file(f'{out_final}\cell_grid.geojson')

In [50]:
start_index = 0
extent = None
# set start index
if not start_index:
    si = -1
else:
    si = start_index

# If an extent shp is provided, it is used
if extent:
    extent = gpd.read_file(args.extent)
    calc_cells = cell.geometry.apply(lambda x: extent.intersects(x).any()).values
else:
    # otherwise find empty cells in parallel
    try:
        # If the empty cells have been calculated they are cached
        calc_cells = np.load(f'{out_folder}\empty_index.npy')
        print("found existing cell index")
    except FileNotFoundError:
        list_of_delayed_functions = []

        def check_cell(Fx, cell):
            try:
                Ax = Fx.rio.clip([cell])
                return not da.all(Ax == da.zeros_like(Ax)).compute()
            except NoDataInBounds:
                return False
            except ValueError:
                return False

        print("Checking empty cells...")
        for i, c in enumerate(cell):
            list_of_delayed_functions.append(dask.delayed(check_cell)(Fx, c))

        with ProgressBar():
            calc_cells = dask.compute(*list_of_delayed_functions)

        calc_cells = [x for x in calc_cells]
        np.save(f'{out_folder}/empty_index.npy', calc_cells)

Checking empty cells...



KeyboardInterrupt



In [49]:
for i, c in enumerate(tqdm(cell.iloc[calc_cells])):
    if i < si:  # start index
        pass
    else:
        try:
            Ax = Fx.rio.clip([c])
            if not da.all(Ax == da.zeros_like(Ax)).compute():
                C_arr = full_inference_numpy(np.asarray(Ax.compute()), model)

                out_C_buf = new_3d_xda(C_arr, Ax)
                out_C_buf = out_C_buf.rio.write_crs(args.crs)

                clipper = c.buffer(-block_buffer, cap_style=3, join_style=2)

                out_C = out_C_buf.rio.clip([clipper])

                out_C = (out_C * (2**args.bit_depth - 1)).astype("uint16")

                out_fname = Path(out_folder) / f"C_{i:04d}.tif"
                save_raster(out_C, out_fname, crs=args.crs)
                print(f"SAVED {i}")
            else:
                print(f"Skip empty {i}")

        except NoDataInBounds:
            print(f"Error in {i}")
        except ValueError:
            print(f"Error in {i}")

# Merge to a vrt file
filelist = list(out_folder.glob("*.tif"))
filelist = [str(x.resolve()) + "\n" for x in filelist]
with open(out_folder / "filelist.txt", "w") as f:
    f.writelines(filelist)

print("Writing .vrt file...")
subprocess.run(
    [
        "gdalbuildvrt",
        "-input_file_list",
        out_folder / "filelist.txt",
        "-a_srs",
        args.crs,
        out_final / f"{input_file.stem}__{model_file.stem}_C.vrt",
    ]
)

NameError: name 'calc_cells' is not defined

In [72]:
from skimage.color import label2rgb

def full_inference_numpy(A, clf):
    # Reshaping
    A0 = np.moveaxis(A,0,2)
    ny, nx, chan = A0.shape
    a = A0.reshape(ny*nx, chan)

    # Classification
    c0 = clf.predict_proba(a)
    c = (254*c0).astype(np.uint8)

    # Inverse reshaping
    C = c.reshape(ny,nx, -1)
    C = np.moveaxis(C,2,0)
 
    return C

def full_inference(A: dask.array, clf) -> dask.array:
    # Reshaping
    A0 = da.moveaxis(A,0,2)
    ny, nx, chan = A0.shape
    a = A0.reshape(ny*nx, chan)

    # Classification
    c0 = clf.predict_proba(a)
    c = (254*c0).astype(np.uint8)

    # Inverse reshaping
    C = c.reshape(ny,nx, -1)
    C = da.moveaxis(C,2,0)
 
    return C

def inference(a: dask.array, clf) -> dask.array:
    c0 = clf.predict_proba(a)
    c = (254*c0).astype(np.uint8)
    return c

def read_masked_data(A: dask.array):
    """ Picks nonzero values along depth from A and returns rows of nonzero values
    and their index mask
    
    Performance depends highly on the mask rechunking size. If the chunk size is too large, memory 
    use per worker is too high, and if it is too low, chunks are probably copied across workers so
    that system memory usage is too high and SLURM job crashes.
    
    params
    A : array
    
    returns
    data: nonzero rows
    mask: row locations
    """

    A0 = da.moveaxis(A,0,2)
    ny, nx, chan = A0.shape
    a = A0.reshape(ny*nx, chan)

    mask = da.where(~da.all(a==0,axis=1))[0]
    mask.compute_chunk_sizes()
    mask = mask.rechunk((1e6,-1)) #inference float chunks get too big otherwise
    
    data = a[mask,:]
    return data, mask

def masked_inference(A: dask.array, clf)->dask.array:
    """Classifies an array depth-wise 
    """
    
    # Masking
    data, mask = read_masked_data(A)
    
    # New data
    chan, ny, nx = A.shape
    c = da.zeros((ny*nx, len(clf.classes_)), dtype=np.uint8) #empty array for results
    
    # Inference
    if len(data)!=0:
        c0 = inference(data, clf)
        c[mask,:] = c0

    # Inverse reshape
    C = c.reshape(ny,nx, -1)
    C = da.moveaxis(C,2,0)
    
    return C

def block_map(x):
    if not da.all(x == da.zeros_like(x)):
        return full_inference(x, clf)
    else:
        return da.zeros((len(classes), x.shape[1], x.shape[2])).astype(np.uint8)

def read_chunks(D: da.Array):
    """Reads chunks and their coordinates from a larger Dask array
    into a list
    """

    chunks = []
    coords = []
    x0 = 0
    x1 = 0
    y0 = 0
    y1 = 0
    # "normal" block shape for coordinate calculation
    base_shape = D.blocks[(0,0,0)].shape
    print("Reading chunks...")
    for inds in tqdm(itertools.product(*map(range, D.blocks.shape)),
                     total=np.product(D.blocks.shape)):
        # read chunk
        chunk = D.blocks[inds]
        chunks.append(chunk)

        # upper left corner
        x0 = (inds[2])*base_shape[2]
        y0 = (inds[1])*base_shape[1]

        # lower right corner
        x1 = x0 + chunk.shape[2]
        y1 = y0 + chunk.shape[1]
        coords.append({'x0':x0,
                       'x1':x1,
                       'y0':y0,
                       'y1':y1})
    return chunks, coords

def chunk_mapping(func, chunks):
    """maps a function to a list of dask arrays
    """
    out_chunks = []
    for chunk in tqdm(chunks):
        out_chunks.append(func(chunk))
    return out_chunks

def preprocess_arrays(X, func):
    """Turns a Xarray DataArray into a Dask array, reads it to chunks
    and applies a function on the chunks
    
    Returns:
        C_list: list of chunks with the function lazily applied
        X_list: list of the corresponding original xarray chunks
    """
    D = X.to_array
    D = da.asarray(D)
    print(D)
    chunks, coords = read_chunks(D)
    C_list = chunk_mapping(func, chunks)
    X_list = [X[:,c['y0']:c['y1'], c['x0']:c['x1']] for c in coords]
    return C_list, X_list



def apply_preprocess_arrays(dataset: xr.Dataset, func):
    """
    Applies preprocess_arrays function to every DataArray in the input Dataset.

    Args:
        dataset: Input xarray Dataset.
        func: Function to be applied on the chunks of each DataArray.

    Returns:
        result_dataset: xarray Dataset with processed DataArrays.
    """
    result_data_vars = {}
    for var_name, data_var in dataset.data_vars.items():
        C_list, X_list = preprocess_arrays(data_var, func)
        result_data_vars[var_name] = xr.concat(C_list, dim='chunk')

    result_dataset = xr.Dataset(result_data_vars)

    return result_dataset

In [73]:
from pathlib import Path
from datetime import datetime
t = datetime.now()
timestamp = t.strftime('%Y-%m-%dT%H-%M-%S')
out_folder = Path(f'tulkinta-{timestamp}')
out_folder_temp = Path(f'tulkinta-{timestamp}-temp')

out_folder.mkdir(exist_ok=True)
out_folder_temp.mkdir(exist_ok=True)
start_i = 0

In [74]:
def produce_aggregate_maps(C):
    S = C.argmax(axis=0).astype(np.uint8)
    M = C.max(axis=0)
    return S,M

In [75]:
new_3d_xda = lambda c, d: xr.DataArray(c, 
                                       name='classification', 
                                       coords={'class': np.arange(len(classes)), 
                                               'y': d.y, 
                                               'x': d.x},
                                       dims=('class', 'y', 'x'))
new_2d_xda = lambda c, d: xr.DataArray(c, 
                                       coords={'y': d.y, 
                                               'x': d.x}, 
                                       dims=('y', 'x'))

def save_raster(x, name):
    if not da.all(x == da.zeros_like(x)):
        x.rio.to_raster(name, 
                       compress='LZW',
                       crs="EPSG:3067",
                       tiled=True,
                       windowed=True)

In [76]:
# Lazily applies the classification function to an xarray. No computation yet
C_list, X_list = preprocess_arrays(cube, block_map)

dask.array<array, shape=(), dtype=object, chunksize=(), chunktype=numpy.ndarray>


IndexError: Too many indices for array

In [58]:
data_array = cube.to_array(dim='band')


In [67]:
data_array[[1]]

<xarray.DataArray (band: 1, y: 53241, x: 38477)>
array([[[[  255.,   255.,   255., ...,   255.,   255.,   255.],
         [  255.,   255.,   255., ...,   255.,   255.,   255.],
         [  255.,   255.,   255., ...,   255.,   255.,   255.],
         ...,
         [  255.,   255.,   255., ...,   255.,   255.,   255.],
         [  255.,   255.,   255., ...,   255.,   255.,   255.],
         [  255.,   255.,   255., ...,   255.,   255.,   255.]]],


       [[[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         ...,
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.]]],


       [[[  127.,   127.,   127., ...,   127.,   127.,   127.],
         [  127.,   127.,   127., ...,   127.,   127.,   127.],
...
         [65535., 65535., 65535., ..., 65535., 65535., 65535.]]],


       [[[65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         ...,
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.]]],


       [[[65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         ...,
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.],
         [65535., 65535., 65535., ..., 65535., 65535., 65535.]]]],
      dtype=float32)
Coordinates:
  * band         (band) object 'ndvi_max' 'dem' ... 's2_b2' 's2_nir'
  * x            (x) float64 2.431e+05 2.431e+05 ... 6.278e+05 6.279e+05
  * y            (y) float64 7.776e+06 7.776e+06 ... 7.244e+06 7.244e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     255
    scale_factor:   1.0
    add_offset:     0.0

In [ ]:
# Predict on whole image and save it as .tif file
# Credits: https://github.com/csc-training/geocomputing/blob/master/machineLearning/02_shallows/05_classification.ipynb
def predictImage(modelName, predictImage):
    #Set file paths for input and output files
    predictedClassesFile = outputImageBase + modelName + '.tif'
    predictedClassesPath = os.path.join(base_folder, predictedClassesFile)
    
    # Read the satellite image
    with rasterio.open(predictImage, 'r') as image_dataset:
        start_time = time.time()    
        
        #Reshape data to 1D as we did before model training
        #image_data = image_dataset.read()
        image_data = image_dataset.read()
        image_data2 = np.transpose(image_data, (1, 2, 0))
        #pixels = image_data2.reshape(-1, 22)
        pixels = image_data2.reshape(-1, 27)
        #Load the model from the saved file
        modelFilePath = os.path.join(base_folder, ('model_' + modelName + '.sav'))
        trained_model = load(modelFilePath)
        
        # predict the class for each pixel
        prediction = trained_model.predict(pixels)
        
        # Reshape back to 2D
        print('Prediction shape in 1D: ', prediction.shape)
        prediction2D = np.reshape(prediction, (image_dataset.meta['height'], image_dataset.meta['width']))
        #prediction2D = np.reshape(prediction, ( 705, 1213))
        print('Prediction shape in 2D: ', prediction2D.shape)
        
        # Save the results as .tif file.
        # Copy the coorindate system information, image size and other metadata from the satellite image 
        outputMeta = image_dataset.meta
        
        # Change the number of bands and data type.
        #outputMeta.update(count=1, dtype='uint8')
        outputMeta.update(count=1, dtype='uint8', nodata=255)
        # Writing the image on the disk
        with rasterio.open(predictedClassesPath, 'w', **outputMeta) as dst:
            dst.write(prediction2D, 1)
        plt.imshow(prediction2D)
        print('Predicting took: ', round((time.time() - start_time), 1), ' seconds')